#### 상대모멘텀
- 10개의 종목을 이용하여 투자 전략 시스템

    1. 월초부터 월말까지의 수정종가를 이용하여 수익율 생성
    2. 월별 수익율이 높은 n개를 선택
    3. 해당하는 종목을 매수하여 해당하는 종목에 신호가 없어지면 매도하는 형식으로 수익율 생성

In [1]:
import pandas as pd
import numpy as np
import os 
from glob import glob
import warnings

In [2]:
# warning 메시지 출력 방지 
warnings.filterwarnings('ignore')

In [8]:
# 월별 수익율을 계산하는 함수 
def create_1m_rtn(_df, 
                  _ticker, 
                  _start = '2010-01-01', 
                  _col = 'Adj Close'):
    # 복사본 생성 
    result = _df.copy()
    # Date 컬럼이 존재하면 index로 변경 
    if 'Date' in result.columns:
        result.set_index('Date', inplace=True)
    # index를 시계열로 변경 
    result.index = pd.to_datetime(result.index)
    # 시작 시간과 기준이 되는 컬럼으로 필터링 
    result = result.loc[_start: , [_col]]
    # 기준 년월 컬럼을 생성 
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1m_rtn'] = 0
    result['CODE'] = _ticker
    # 기준년월(STD-YM)의 유니크한 리스트를 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [9]:
aapl = pd.read_csv("../../csv/AAPL.csv")

In [10]:
sample_aapl, ym_list = create_1m_rtn(aapl, 'AAPL')

In [17]:
sample_aapl.loc[
    sample_aapl['STD-YM'] == '2010-01'
]

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL
2010-01-11,26.294640,2010-01,0,AAPL
2010-01-12,25.995537,2010-01,0,AAPL
2010-01-13,26.362217,2010-01,0,AAPL
2010-01-14,26.209538,2010-01,0,AAPL


In [ ]:
ym_list

In [ ]:
# listdir() 파일의 이름의 목록
os.listdir('./data')
# glob() 파일의 경로와 이름의 목록록
files = glob('./data/*.csv')

In [35]:
# 반복문을 이용

# 종복별 데이터프레임 생성

# 월말의 데이터를 모아주는 작업 
# 월별 수익율 계산
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

for file in files:
    # file : 주식데이터의 경로와 파일명
    # 경로와 파일명을 나눠준다. 
    folder, name = os.path.split(file)
    # name : AAPL.csv --> AAPL    .csv 나눈다. 
    head, tail = os.path.splitext(name)
    # head 변수는 create_1m_rtn() 함수에서 _ticker 사용

    # csv 파일을 로드 
    read_df = pd.read_csv(file)

    # create_1m_rtn() 함수를 호출
    price_df, ym_list = create_1m_rtn(read_df, _ticker = head)

    # price_df 를 stock_df에 단순 행결합 모두 결합
    stock_df = pd.concat( [stock_df, price_df] )

    # 두번째 반복문 생성
    # price_df의 월별 수익율 계산하는 반복문
    for ym in ym_list:
        # ym : 기준년월
        flag = price_df['STD-YM'] == ym
        # 월초 수정 종가
        buy = price_df.loc[flag].iloc[0, 0]
        # 월말 수정 종가
        sell = price_df.loc[flag].iloc[-1, 0]
        # 수익율 계산
        m_rtn = sell / buy
        # price_df에 flag 조건과 1m_rtn 컬럼에 대입
        price_df.loc[flag, '1m_rtn'] = m_rtn
        # 월말의 데이터를 month_last_df에 단순 행 결합
        last_data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat( [month_last_df, last_data] )

In [102]:
# 데이터 백업
month_rtn_df =  month_last_df.copy()

In [103]:
# 인덱스를 리셋
month_rtn_df.reset_index(inplace=True)

In [104]:
# pivot_table을 이용하여 테이블을 재구조화
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date', 
    columns= 'CODE', 
    values= '1m_rtn'
)

In [105]:
# rank 함수를 이용하여 rank를 퍼센트로 표현
month_rtn_df = month_rtn_df.rank(
    axis = 1, 
    ascending= False, 
    method= 'max', 
    pct= True
)

In [106]:
# 상위의 15% 종목만 선택
# where( 조건식 ,  조건식이 거짓인 경우 변경할 데이터값 )
month_rtn_df = month_rtn_df.where(month_rtn_df < 0.40, 0)

In [107]:
# 0이 아닌 데이터는 1로 변환
month_rtn_df[month_rtn_df != 0] = 1

In [108]:
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2010-02-26,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2010-03-31,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2010-04-30,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2010-05-28,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2019-03-29,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2019-04-30,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [109]:
# CODE 값들로 이루어진 리스트 생성
stock_codes = stock_df['CODE'].unique()
# month_rtn_df.columns

In [110]:
month_rtn_df.loc['2010-01-29', month_rtn_df.loc['2010-01-29'] == 1].index

Index(['BND', 'GLD', 'SPY'], dtype='object', name='CODE')

In [111]:
sig_dict = dict()

for date in month_rtn_df.index:
    # date : 인덱스의 값들
    ticker_list = list(
        month_rtn_df.loc[date, month_rtn_df.loc[date] == 1].index
    )
    sig_dict[date] = ticker_list

In [112]:
sig_dict

{Timestamp('2010-01-29 00:00:00'): ['BND', 'GLD', 'SPY'],
 Timestamp('2010-02-26 00:00:00'): ['AAPL', 'GDX', 'SPY'],
 Timestamp('2010-03-31 00:00:00'): ['AAPL', 'AMZN', 'USM'],
 Timestamp('2010-04-30 00:00:00'): ['AAPL', 'GDX', 'MSFT'],
 Timestamp('2010-05-28 00:00:00'): ['BND', 'GDX', 'GLD'],
 Timestamp('2010-06-30 00:00:00'): ['BND', 'GDX', 'GLD'],
 Timestamp('2010-07-30 00:00:00'): ['MSFT', 'SPY', 'USM'],
 Timestamp('2010-08-31 00:00:00'): ['GDX', 'GLD', 'SLV'],
 Timestamp('2010-09-30 00:00:00'): ['AAPL', 'AMZN', 'SLV'],
 Timestamp('2010-10-29 00:00:00'): ['AMZN', 'MSFT', 'SLV'],
 Timestamp('2010-11-30 00:00:00'): ['AMZN', 'GDX', 'SLV'],
 Timestamp('2010-12-31 00:00:00'): ['MSFT', 'SLV', 'USM'],
 Timestamp('2011-01-31 00:00:00'): ['AAPL', 'BND', 'SPY'],
 Timestamp('2011-02-28 00:00:00'): ['GDX', 'GLD', 'SLV'],
 Timestamp('2011-03-31 00:00:00'): ['AMZN', 'SLV', 'USM'],
 Timestamp('2011-04-29 00:00:00'): ['AMZN', 'GLD', 'SLV'],
 Timestamp('2011-05-31 00:00:00'): ['AAPL', 'BND', 'USM']

In [113]:
stock_df.head(2)

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL


In [114]:
# 거래내역 컬럼을 생성하는 함수 
def create_trade_book(_df, _code):
    # 복사본 생성
    book = _df[_code].copy()
    book['STD-YM'] = book.index.strftime('%Y-%m')
    for c in _code:
        book[f"p_{c}"] = ''
        book[f'r_{c}'] = ''
    return book

In [115]:
# stock_df를 재구조화
stock_c_matrix = stock_df.reset_index().pivot_table(
    index = 'Date', 
    columns= 'CODE', 
    values= stock_df.columns[0]
)


In [116]:
stock_c_matrix

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-04,26.782711,133.899994,60.611969,44.908779,109.800003,NaN,24.525019,17.230000,93.675278,36.015179
2010-01-05,26.829010,134.690002,60.789135,45.341774,109.699997,NaN,24.532942,17.510000,93.923241,35.998024
2010-01-06,26.402260,132.250000,60.766037,46.443077,111.510002,NaN,24.382378,17.860001,93.989357,35.680672
2010-01-07,26.353460,130.000000,60.719822,46.217175,110.820000,NaN,24.128809,17.889999,94.386139,35.208931
2010-01-08,26.528664,133.520004,60.781410,46.913723,111.370003,NaN,24.295214,18.150000,94.700218,34.651424
...,...,...,...,...,...,...,...,...,...,...
2019-06-18,198.449997,1901.369995,82.397118,23.670000,127.120003,36.700001,135.160004,14.050000,290.984741,50.540001
2019-06-19,197.869995,1908.790039,82.676468,24.000000,127.889999,36.779999,135.690002,14.170000,291.641541,50.040001
2019-06-20,199.460007,1918.189941,82.806168,25.049999,131.110001,36.959999,136.949997,14.450000,294.427979,49.320000


In [117]:
book = create_trade_book(stock_c_matrix, stock_codes)

In [118]:
book.columns

Index(['AAPL', 'AMZN', 'BND', 'GDX', 'GLD', 'GM', 'MSFT', 'SLV', 'SPY', 'USM',
       'STD-YM', 'p_AAPL', 'r_AAPL', 'p_AMZN', 'r_AMZN', 'p_BND', 'r_BND',
       'p_GDX', 'r_GDX', 'p_GLD', 'r_GLD', 'p_GM', 'r_GM', 'p_MSFT', 'r_MSFT',
       'p_SLV', 'r_SLV', 'p_SPY', 'r_SPY', 'p_USM', 'r_USM'],
      dtype='object', name='CODE')

In [119]:
book.head(2)

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM,...,p_GM,r_GM,p_MSFT,r_MSFT,p_SLV,r_SLV,p_SPY,r_SPY,p_USM,r_USM
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,26.782711,133.899994,60.611969,44.908779,109.800003,NaN,24.525019,17.23,93.675278,36.015179,...,,,,,,,,,,
2010-01-05,26.829010,134.690002,60.789135,45.341774,109.699997,NaN,24.532942,17.51,93.923241,35.998024,...,,,,,,,,,,


In [120]:
sig_dict.items()

dict_items([(Timestamp('2010-01-29 00:00:00'), ['BND', 'GLD', 'SPY']), (Timestamp('2010-02-26 00:00:00'), ['AAPL', 'GDX', 'SPY']), (Timestamp('2010-03-31 00:00:00'), ['AAPL', 'AMZN', 'USM']), (Timestamp('2010-04-30 00:00:00'), ['AAPL', 'GDX', 'MSFT']), (Timestamp('2010-05-28 00:00:00'), ['BND', 'GDX', 'GLD']), (Timestamp('2010-06-30 00:00:00'), ['BND', 'GDX', 'GLD']), (Timestamp('2010-07-30 00:00:00'), ['MSFT', 'SPY', 'USM']), (Timestamp('2010-08-31 00:00:00'), ['GDX', 'GLD', 'SLV']), (Timestamp('2010-09-30 00:00:00'), ['AAPL', 'AMZN', 'SLV']), (Timestamp('2010-10-29 00:00:00'), ['AMZN', 'MSFT', 'SLV']), (Timestamp('2010-11-30 00:00:00'), ['AMZN', 'GDX', 'SLV']), (Timestamp('2010-12-31 00:00:00'), ['MSFT', 'SLV', 'USM']), (Timestamp('2011-01-31 00:00:00'), ['AAPL', 'BND', 'SPY']), (Timestamp('2011-02-28 00:00:00'), ['GDX', 'GLD', 'SLV']), (Timestamp('2011-03-31 00:00:00'), ['AMZN', 'SLV', 'USM']), (Timestamp('2011-04-29 00:00:00'), ['AMZN', 'GLD', 'SLV']), (Timestamp('2011-05-31 00:00:

In [121]:
for date, values in sig_dict.items():
    # print(date)
    # print(values)
    for stock in values:
        # stock : 종목명
        book.loc[date, f"p_{stock}"] = f"ready_{stock}"

In [122]:
book['p_AMZN'].value_counts()

p_AMZN
              2337
ready_AMZN      47
Name: count, dtype: int64

In [123]:
# 거래내역 추가 
def trading(_df, s_codes):
    std_ym = ''
    buy_phase = False
    _book = _df.copy()

    # 종목 별로 순회 (반복)
    for code in s_codes:
        # 인덱스별 반목 
        for idx in _book.index:
            # 특정 종목 코드에서 포지션을 잡는다.
            if (_book.loc[idx, f"p_{code}"] == "") & \
                (_book.shift().loc[idx, f"p_{code}"] == f"ready_{code}"):
                std_ym = _book.loc[idx, 'STD-YM']
                buy_phase = True
            # 해당 종목 코드에서 신호가 잡혀있다면 매수 유지 
            if (_book.loc[idx, f"p_{code}"] == '') & \
                (_book.loc[idx, 'STD-YM'] == std_ym) & \
                (buy_phase):
                _book.loc[idx, f"p_{code}"] = f"buy_{code}"
            # buy_phase, std_ym 초기화
            if _book.loc[idx, f"p_{code}"] == '':
                std_ym = ''
                buy_phase = False
    return _book

In [124]:
book2 = trading(book, stock_codes)

In [125]:
book2['p_SPY'].value_counts()

p_SPY
             1822
buy_SPY       536
ready_SPY      26
Name: count, dtype: int64

In [126]:
book2.head(1)

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM,...,p_GM,r_GM,p_MSFT,r_MSFT,p_SLV,r_SLV,p_SPY,r_SPY,p_USM,r_USM
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,26.782711,133.899994,60.611969,44.908779,109.800003,NaN,24.525019,17.23,93.675278,36.015179,...,,,,,,,,,,


In [127]:
# 수익율 계산 함수 
def multi_return(_df, s_codes):
    # 복사본 생성 
    _book = _df.copy()
    rtn = 1
    buy_dict = dict()
    sell_dict = dict()

    for idx in _book.index:
        for code in s_codes:
            # 매수 타이밍 -> 2일전에 '' 1일전에 'ready' 오늘이 'buy'
            if (_book.shift(2).loc[idx, f"p_{code}"] == '') & \
                (_book.shift(1).loc[idx, f"p_{code}"] == f"ready_{code}") & \
                (_book.loc[idx, f"p_{code}"] == f"buy_{code}"):
                # 해당하는 날짜의 특정 종목의 수정종가
                buy_dict[code] = _book.loc[idx, code]
                print(f"매수일 : {idx}, 매수가 : {buy_dict[code]}, 종목코드 : {code}")
            # 매도 타이밍 -> 1일 전에 'buy' 오늘이 ''
            elif (_book.shift(1).loc[idx, f'p_{code}'] == f'buy_{code}') & \
                (_book.loc[idx, f'p_{code}'] == ''):
                sell_dict[code] = _book.loc[idx, code]
                # 수익율 계산
                rtn = sell_dict[code] / buy_dict[code]
                _book.loc[idx, f"r_{code}"] = rtn
                print(f"매도일 : {idx}, 매도가 : {sell_dict[code]}, 종목코드 : {code}, 수익율 : {rtn}")
            # buy_dict, sell_dict 초기화
            if _book.loc[idx, f"p_{code}"] == "":
                buy_dict[code] = 0
                sell_dict[code] = 0
    return _book

In [128]:
rtn_book = multi_return(book2, stock_codes)

매수일 : 2010-02-01 00:00:00, 매수가 : 61.280487, 종목코드 : BND
매수일 : 2010-02-01 00:00:00, 매수가 : 108.349998, 종목코드 : GLD
매수일 : 2010-02-01 00:00:00, 매수가 : 90.145805, 종목코드 : SPY
매수일 : 2010-03-01 00:00:00, 매수가 : 26.154476, 종목코드 : AAPL
매도일 : 2010-03-01 00:00:00, 매도가 : 61.585163, 종목코드 : BND, 수익율 : 1.0049718273289832
매수일 : 2010-03-01 00:00:00, 매수가 : 42.019035, 종목코드 : GDX
매도일 : 2010-03-01 00:00:00, 매도가 : 109.43, 종목코드 : GLD, 수익율 : 1.0099677159200318
매수일 : 2010-04-01 00:00:00, 매수가 : 131.809998, 종목코드 : AMZN
매도일 : 2010-04-01 00:00:00, 매도가 : 43.675705, 종목코드 : GDX, 수익율 : 1.0394266550862008
매도일 : 2010-04-01 00:00:00, 매도가 : 97.770996, 종목코드 : SPY, 수익율 : 1.0845873083056943
매수일 : 2010-04-01 00:00:00, 매수가 : 36.212452, 종목코드 : USM
매도일 : 2010-05-03 00:00:00, 매도가 : 137.490005, 종목코드 : AMZN, 수익율 : 1.043092383629351
매수일 : 2010-05-03 00:00:00, 매수가 : 46.951374, 종목코드 : GDX
매수일 : 2010-05-03 00:00:00, 매수가 : 24.56805, 종목코드 : MSFT
매도일 : 2010-05-03 00:00:00, 매도가 : 36.744232, 종목코드 : USM, 수익율 : 1.0146850039317967
매도일 : 2010-06-01 

In [129]:
# 누적 수익율 계산
def multi_acc_return(_df, s_codes):
    _book = _df.copy()
    acc_rtn = 1

    for idx in _book.index:
        count = 0
        rtn = 0
        for code in s_codes:
            # 수익율 데이터가 존재하는가?
            if _book.loc[idx, f"r_{code}"]:
                count += 1
                rtn += _book.loc[idx, f"r_{code}"]
        if(rtn != 0) & (count != 0):
            acc_rtn *= (rtn / count)
            print(f"누적 - 매도일 : {idx}, 매도 종목수 : {count}, 수익율 : {round( rtn / count, 2 )}")
        _book.loc[idx, 'acc_rtn'] = acc_rtn
    print(f"총 누적 수익율은 {acc_rtn}입니다")
    return _book, acc_rtn

In [130]:
multi_acc_return(rtn_book, stock_codes)

누적 - 매도일 : 2010-03-01 00:00:00, 매도 종목수 : 2, 수익율 : 1.01
누적 - 매도일 : 2010-04-01 00:00:00, 매도 종목수 : 2, 수익율 : 1.06
누적 - 매도일 : 2010-05-03 00:00:00, 매도 종목수 : 2, 수익율 : 1.03
누적 - 매도일 : 2010-06-01 00:00:00, 매도 종목수 : 2, 수익율 : 1.05
누적 - 매도일 : 2010-08-02 00:00:00, 매도 종목수 : 3, 수익율 : 0.98
누적 - 매도일 : 2010-09-01 00:00:00, 매도 종목수 : 3, 수익율 : 0.93
누적 - 매도일 : 2010-10-01 00:00:00, 매도 종목수 : 2, 수익율 : 1.07
누적 - 매도일 : 2010-11-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.08
누적 - 매도일 : 2010-12-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.97
누적 - 매도일 : 2011-01-03 00:00:00, 매도 종목수 : 2, 수익율 : 1.11
누적 - 매도일 : 2011-02-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.15
누적 - 매도일 : 2011-03-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.01
누적 - 매도일 : 2011-04-01 00:00:00, 매도 종목수 : 2, 수익율 : 0.99
누적 - 매도일 : 2011-05-02 00:00:00, 매도 종목수 : 1, 수익율 : 0.94
누적 - 매도일 : 2011-06-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.04
누적 - 매도일 : 2011-07-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.0
누적 - 매도일 : 2011-08-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.02
누적 - 매도일 : 2011-09-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.96
누적 - 매도일 : 

(CODE              AAPL         AMZN        BND        GDX         GLD  \
 Date                                                                    
 2010-01-04   26.782711   133.899994  60.611969  44.908779  109.800003   
 2010-01-05   26.829010   134.690002  60.789135  45.341774  109.699997   
 2010-01-06   26.402260   132.250000  60.766037  46.443077  111.510002   
 2010-01-07   26.353460   130.000000  60.719822  46.217175  110.820000   
 2010-01-08   26.528664   133.520004  60.781410  46.913723  111.370003   
 ...                ...          ...        ...        ...         ...   
 2019-06-18  198.449997  1901.369995  82.397118  23.670000  127.120003   
 2019-06-19  197.869995  1908.790039  82.676468  24.000000  127.889999   
 2019-06-20  199.460007  1918.189941  82.806168  25.049999  131.110001   
 2019-06-21  198.779999  1911.300049  82.576698  25.209999  131.979996   
 2019-06-24  199.169998  1907.953857  82.726349  25.703501  133.501907   
 
 CODE               GM        MSFT  